In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.7 MB/s eta 0:11:11tcmalloc: large alloc 1147494400 bytes == 0x562bc1526000 @  0x7f3ef469b615 0x562b8820b4cc 0x562b882eb47a 0x562b8820e2ed 0x562b882ffe1d 0x562b88281e99 0x562b8827c9ee 0x562b8820fbda 0x562b88281d00 0x562b8827c9ee 0x562b8820fbda 0x562b8827e737 0x562b88300c66 0x562b8827ddaf 0x562b88300c66 0x562b8827ddaf 0x562b88300c66 0x562b8827ddaf 0x562b88210039 0x562b88253409 0x562b8820ec52 0x562b88281c25 0x562b8827c9ee 0x562b8820fbda 0x562b8827e737 0x562b8827c9ee 0x562b8820fbda 0x562b8827d915 0x562b8820fafa 0x562b8827dc0d 0x562b8827c9ee
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:10:44tcmalloc: large alloc 1434370048 bytes == 0x562c05b7c000 @  0x7f3ef469b615 0x562b8820b4cc 0x562b882eb47a 0x562b8820e2ed 0x562b882ffe1d 0x562b88281e99 0x562b8827c9ee 0x562b8820fbda 0x562b88281d00 0x562b8827c9ee 0x562b8820fbda 0x562b8827e737 0x562b88300c66 0x562b8827dd

In [ ]:
!pip install transformers requests beautifulsoup4

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [6]:
# Instantiate model
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [15]:
# Encode and calculate sentiment
tokens = tokenizer.encode("I hate this, absolutely worst", return_tensors='pt')
print(len(tokens))

1


In [14]:
print(tokens[0])

tensor([  101,   151, 39487, 10372,   117, 35925, 10563, 43060,   102])


In [10]:
tokenizer.decode(tokens[0])

'[CLS] i hate this, absolutely worst [SEP]'

In [17]:
result = model(tokens)
result

SequenceClassifierOutput([('logits',
                           tensor([[ 5.0982,  1.7063, -0.9308, -2.9229, -2.0965]],
                                  grad_fn=<AddmmBackward>))])

In [19]:
torch.argmax(result.logits)

tensor(0)

In [23]:
sent = "As I mentioned before, the way extractive question answering works is by computing the best start and end indexes for where the answer is located in the context."
tokens = tokenizer.encode(sent, return_tensors='pt')
tokens

tensor([[  101, 10146,   151, 24838, 11364,   117, 10103, 12140, 17708, 34803,
         18722, 42942, 10285, 13144, 10127, 10151, 42283, 10103, 11146, 13982,
         10110, 11421, 10534, 10165, 10139, 10982, 10103, 42942, 10127, 12120,
         10104, 10103, 27195,   119,   102]])

In [24]:
result = model(tokens)
result

SequenceClassifierOutput([('logits',
                           tensor([[-2.8926, -2.0695,  0.0960,  1.9847,  2.1428]],
                                  grad_fn=<AddmmBackward>))])

In [25]:
torch.argmax(result.logits)

tensor(4)

In [26]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [30]:
len(reviews)

11

In [41]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont?start=60')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results2 = soup.find_all('p', {'class':regex})
reviews2 = [result2.text for result2 in results2]

In [42]:
reviews2

['Menu has changed and most of breakfast is not available when I was trying to order. It should be updated. It is very hard to make decision without pictures.',
 'This place is awesome. Very relaxed atmosphere, extremely quick service, and the meal was on point. I had their flat white and The Social Brekkie roll. Interesting with the mix of savory and sweetness on the Brekkie Roll. Will be coming back here for sure!',
 "Wow. This is it. This is the place to go. Turned up for breakfast after reading all the reviews and they weren't lying. Great coffee. Attentive and friendly staff. Food to die for. Location that's perfect for people watching (hey, we all need hobbies). All of us are completely satisfied with what we had fire brekkie, and so reasonably priced too! Forget the rest, this is the best. Run don't walk. Very on down to the best place in town. You can come up with your own catchy slogan from here. Five stars. Nuff said.",
 'Great Ambiance, friendly service. Overall a winner. Go

In [31]:
results[0]

<p class="comment__09f24__gu0rG css-1sufhje"><span class=" raw__09f24__T4Ezm" lang="en">I went here a little while ago- a beautiful morning,<br/>a lovely little brew house on a quaint street corner- perfection.<br/><br/>I went to this cafe with my step-daughter Lucille.<br/>She was always raving about how great it was to her mother, so I thought it would be a nice idea to go here with her for her birthday... boy was I wrong.<br/><br/>She announced her hatred for me while I was waiting for my extra large iced frappé. It felt like hours of awkward silence once she said those four words; "you're a low-life."<br/><br/>Was it in my mind, or was my drink taking ages to arrive? The hands on the clock didn't budge from the last time I glanced at them- 7:43AM, where the fuck is my drink?<br/><br/>"Why do you always feel you have to be my friend? You're not my dad!" She fired.<br/><br/>I could only sit there, my head facing down towards the floral tablecloth that lay beneath my quivering arms. T

In [39]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
df

,review
0,I went here a little while ago- a beautiful mo...
1,I came to Social brew cafe for brunch while ex...
2,Ricotta hot cakes! These were so yummy. I ate ...
3,Good coffee and toasts. Straight up and down -...
4,We came for brunch twice in our week-long visi...
5,Ron & Jo are on the go down under and Wow! We...
6,Great coffee and vibe. That's all you need. C...
7,Great coffee and vibe. That's all you need. C...
8,This cafe is one of the most popular cafes whe...
9,This place is a gem. The ambiance is to die fo...


In [43]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [45]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,I went here a little while ago- a beautiful mo...,2
1,I came to Social brew cafe for brunch while ex...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,Good coffee and toasts. Straight up and down -...,5
4,We came for brunch twice in our week-long visi...,4
5,Ron & Jo are on the go down under and Wow! We...,5
6,Great coffee and vibe. That's all you need. C...,5
7,Great coffee and vibe. That's all you need. C...,4
8,This cafe is one of the most popular cafes whe...,5
9,This place is a gem. The ambiance is to die fo...,3


In [57]:
df['review'][0]

'I went here a little while ago- a beautiful morning,a lovely little brew house on a quaint street corner- perfection.I went to this cafe with my step-daughter Lucille.She was always raving about how great it was to her mother, so I thought it would be a nice idea to go here with her for her birthday... boy was I wrong.She announced her hatred for me while I was waiting for my extra large iced frappé. It felt like hours of awkward silence once she said those four words; "you\'re a low-life."Was it in my mind, or was my drink taking ages to arrive? The hands on the clock didn\'t budge from the last time I glanced at them- 7:43AM, where the fuck is my drink?"Why do you always feel you have to be my friend? You\'re not my dad!" She fired.I could only sit there, my head facing down towards the floral tablecloth that lay beneath my quivering arms. The bullet lodged in my heart.I don\'t understand why she hates me so much; is it my jokes? The funny way I walk? The fact that I often scream my

In [59]:
df['review'][1]

'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.'

In [65]:
r = "This is really worst, pathetic"
token = tokenizer.encode(r, return_tensors='pt')
result = model(tokens)
int(torch.argmax(result.logits)) + 1

5